In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve

## Bootstrap Function

In [2]:

def get_optimal_f1_threshold(y_true, y_pred):
    epsilon = 1e-10
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    f1 = 2 * precision * recall / (precision + recall + epsilon)
    return thresholds[np.argmax(f1)]

def bootstrap_stats(y_true, y_prob1, y_prob2, bootstrap_samples=10000, y_true2=None):
    test_sets_different = y_true2 is not None
    if test_sets_different:
        y_true2 = np.array(y_true2)
    else:
        y_true2 = y_true
    y_true = np.array(y_true)

    results = {}
    results['auc1'] = roc_auc_score(y_true, y_prob1)
    results['auc2'] = roc_auc_score(y_true2, y_prob2)
    results['auc_diff'] = results['auc2'] - results['auc1']
    aucs1 = []
    aucs2 = []
    auc_diffs = []
    thresholds1 = get_optimal_f1_threshold(y_true, y_prob1)
    thresholds2 = get_optimal_f1_threshold(y_true2, y_prob2)
    results['f1_score1'] = f1_score(y_true, y_prob1 > thresholds1)
    results['f1_score2'] = f1_score(y_true2, y_prob2 > thresholds2)
    results['f1_diff'] = results['f1_score2'] - results['f1_score1']
    f1_scores1 = []
    f1_scores2 = []
    f1_diffs = []

    # Run the bootstrap
    for _ in tqdm(range(bootstrap_samples)):
        idx1 = np.random.choice(range(len(y_true)), len(y_true), replace=True)
        if test_sets_different:
            idx2 = np.random.choice(range(len(y_true2)), len(y_true2), replace=True)
        else:
            idx2 = idx1
        auc1 = roc_auc_score(y_true[idx1], y_prob1[idx1])
        auc2 = roc_auc_score(y_true2[idx2], y_prob2[idx2])
        thresholds1 = get_optimal_f1_threshold(y_true[idx1], y_prob1[idx1])
        thresholds2 = get_optimal_f1_threshold(y_true2[idx2], y_prob2[idx2])
        f1_1 = f1_score(y_true[idx1], y_prob1[idx1] > thresholds1)
        f1_2 = f1_score(y_true2[idx2], y_prob2[idx2] > thresholds2)
        aucs1.append(auc1)
        aucs2.append(auc2)
        auc_diffs.append(auc2 - auc1)
        f1_scores1.append(f1_1)
        f1_scores2.append(f1_2)
        f1_diffs.append(f1_2 - f1_1)
    
    # Compute confidence intervals
    aucs1 = np.array(aucs1)
    aucs2 = np.array(aucs2)
    auc_diffs = np.array(auc_diffs)
    results['auc_diff_ci'] = np.percentile(auc_diffs, [2.5, 97.5])
    if results['auc2'] > results['auc1']:
        results['auc_diff_p_value'] = (auc_diffs < 0).mean()
    else:
        results['auc_diff_p_value'] = (auc_diffs > 0).mean()
    results['auc1_ci'] = np.percentile(aucs1, [2.5, 97.5])
    results['auc2_ci'] = np.percentile(aucs2, [2.5, 97.5])
    f1_scores1 = np.array(f1_scores1)
    f1_scores2 = np.array(f1_scores2)
    f1_diffs = np.array(f1_diffs)
    results['f1_diff_ci'] = np.percentile(f1_diffs, [2.5, 97.5])
    if results['f1_score2'] > results['f1_score1']:
        results['f1_diff_p_value'] = (f1_diffs < 0).mean()
    else:
        results['f1_diff_p_value'] = (f1_diffs > 0).mean()
    results['f1_score1_ci'] = np.percentile(f1_scores1, [2.5, 97.5])
    results['f1_score2_ci'] = np.percentile(f1_scores2, [2.5, 97.5])
    return results

## Load BRSet Labels

In [3]:
brset_embed = pd.read_csv('embeddings.csv') # From Embeddings archive
brset_split = pd.read_csv('split.csv') # generated from resplit_data.ipynb

# Load the true values
y_test = np.array(brset_embed[brset_split['split'] == 'test']['DR_2'])
y_test_embed = np.array(brset_embed[brset_split['embeddings_split'] == 'test']['DR_2'])

In [4]:
model_summary_list = []
difference_summary_list = []

def add_model_line(model_name, summary_list, results, idx):
    summary_list.append({
        'model': model_name,
        'auc': results[f'auc{idx}'],
        'auc_ci' : results[f'auc{idx}_ci'],
        # 'auc_ci_lower': results[f'auc{idx}_ci'][0],
        # 'auc_ci_upper': results[f'auc{idx}_ci'][1],
        'f1_score': results[f'f1_score{idx}'],
        'f1_score_ci': results[f'f1_score{idx}_ci'],
        # 'f1_score_ci_lower': results[f'f1_score{idx}_ci'][0],
        # 'f1_score_ci_upper': results[f'f1_score{idx}_ci'][1]
    })

def add_difference_line(model1_name, model2_name, summary_list, results):
    summary_list.append({
        'model1': model1_name,
        'model1_auc': results['auc1'],
        'model1_f1_score': results['f1_score1'],
        'model2': model2_name,
        'model2_auc': results['auc2'],
        'model2_f1_score': results['f1_score2'],
        'auc_diff': results['auc_diff'],
        'auc_diff_ci': results['auc_diff_ci'],
        # 'auc_diff_ci_lower': results['auc_diff_ci'][0],
        # 'auc_diff_ci_upper': results['auc_diff_ci'][1],
        'auc_diff_p_value': results['auc_diff_p_value'],
        'f1_diff': results['f1_diff'],
        'f1_diff_ci': results['f1_diff_ci'],
        # 'f1_diff_ci_lower': results['f1_diff_ci'][0],
        # 'f1_diff_ci_upper': results['f1_diff_ci'][1],
        'f1_diff_p_value': results['f1_diff_p_value']
    })

In [5]:
bootstrap_samples = 10000 # Number of bootstrap samples to run; 10000 for final results

## Note Analysis

### XGBoost All columns vs Patient History
The all columns represents using all the data that was used to generate the text embeddings data, while the history columns are limited to the columns representing patient history.  The difference between these two represents the effect that adding in a clinical examination of the eye by an ophthalmologist.

In [6]:
xgb_all_columns_test_probs = np.load('probs/xgb_all_columns_test_probs.npy')
xgb_pt_history_test_probs = np.load('probs/xgb_pt_history_test_probs.npy')

results = bootstrap_stats(y_test, xgb_all_columns_test_probs, xgb_pt_history_test_probs, bootstrap_samples=bootstrap_samples)
add_model_line('XGBoost Complete Data', model_summary_list, results, 1)
add_model_line('XGBoost Patient History', model_summary_list, results, 2)
add_difference_line('XGBoost Complete Data', 'XGBoost Patient History', difference_summary_list, results)

100%|██████████| 10000/10000 [00:36<00:00, 272.60it/s]


### XGBoost All columns vs Text Only Embeddings model

This comparison is attempting to assess how well the text embeddings can perform in comparison to using the raw tabular data.  This is not quite as good as doing something like the ConvNextV2 model comparison, as we don't have a language model trained to perform predictions on raw text notes, but it's what we have available.

In [7]:
text_only_test_probs = np.load('probs/text_only_test_probs.npy')

results = bootstrap_stats(y_test, xgb_all_columns_test_probs, text_only_test_probs, bootstrap_samples=bootstrap_samples)

add_model_line('Text Only Model', model_summary_list, results, 2)
add_difference_line('XGBoost Complete Data', 'Text Only Model', difference_summary_list, results)

100%|██████████| 10000/10000 [00:37<00:00, 266.90it/s]


## Image Analysis

### Compare convnextv2-base model with image embeddings model

This comparison gives an idea of what performance can be achieved by using image embeddings vs using a model that was trained on the images directly.  Essentially this tells us how much we are losing by using embeddings instead of the images themselves.

In [8]:
convnextv2_test_emsplit_probs = np.load('probs/convnextv2-base_test_emsplit_probs.npy')
image_only_test_emsplit_probs = np.load('probs/image_only_test_emsplit_probs.npy')

results = bootstrap_stats(y_test_embed, convnextv2_test_emsplit_probs, image_only_test_emsplit_probs, bootstrap_samples=bootstrap_samples)
add_model_line('ConvNetV2 (Embeddings Split)', model_summary_list, results, 1)
add_model_line('Image Only Model (Embeddings Split)', model_summary_list, results, 2)
add_difference_line('ConvNetV2 (Embeddings Split)', 'Image Only Model (Embeddings Split)', difference_summary_list, results)

100%|██████████| 10000/10000 [00:37<00:00, 269.39it/s]


### Compare image only embeddings using embeddings split with split that doesn't have overlapping patients.

The goal here is to see if there is an effect of data leakage, since the original split had 84% of patients in the training set, while the new split has no test patients in the training or validation set.

In [9]:
image_only_test_probs = np.load('probs/image_only_test_probs.npy')

results = bootstrap_stats(y_test, image_only_test_probs, image_only_test_emsplit_probs, bootstrap_samples=bootstrap_samples, y_true2=y_test_embed)
add_model_line('Image Only Model', model_summary_list, results, 1)

add_difference_line('Image Only Model (Disjoint Test Split)', 'Image Only Model (Embeddings Split)', difference_summary_list, results)

100%|██████████| 10000/10000 [00:38<00:00, 257.18it/s]


## Fusion Model Comparisons
In this section we compare the various fusion models with the image only embedding models to see each how each fusion model compares to the image only embeddings.  Basically we're trying to see which model gives the best improvement over using image data alone.

In [10]:
# Early Fusion Model
early_fusion_test_probs = np.load('probs/early_fusion_test_probs.npy')
results = bootstrap_stats(y_test, image_only_test_probs, early_fusion_test_probs, bootstrap_samples=bootstrap_samples)
add_model_line('Early Fusion Model', model_summary_list, results, 2)
add_difference_line('Image Only Model', 'Early Fusion Model', difference_summary_list, results)

100%|██████████| 10000/10000 [00:39<00:00, 256.16it/s]


In [11]:
# Late Fusion Model
late_fusion_test_probs = np.load('probs/late_fusion_test_probs.npy')
results = bootstrap_stats(y_test, image_only_test_probs, late_fusion_test_probs, bootstrap_samples=bootstrap_samples)
add_model_line('Late Fusion Model', model_summary_list, results, 2)
add_difference_line('Image Only Model', 'Late Fusion Model', difference_summary_list, results)

100%|██████████| 10000/10000 [00:38<00:00, 260.66it/s]


In [12]:
# Attention Fusion Model
attention_fusion_test_probs = np.load('probs/attention_fusion_test_probs.npy')
results = bootstrap_stats(y_test, image_only_test_probs, attention_fusion_test_probs, bootstrap_samples=bootstrap_samples)
add_model_line('Attention Fusion Model', model_summary_list, results, 2)
add_difference_line('Image Only Model', 'Attention Fusion Model', difference_summary_list, results)

100%|██████████| 10000/10000 [00:38<00:00, 259.25it/s]


In [13]:
# Early vs Late Fusion Model
# Both of these models do better than the Image Only model; is there a difference between them?
results = bootstrap_stats(y_test, early_fusion_test_probs, late_fusion_test_probs, bootstrap_samples=bootstrap_samples)
add_difference_line('Early Fusion Model', 'Late Fusion Model', difference_summary_list, results)

100%|██████████| 10000/10000 [00:37<00:00, 264.86it/s]


## Summary of Results

Note that any p value of 0 should be interpreted as $p < 1/\text{bootstrap\_samples}$.

In [14]:
model_summary_df = pd.DataFrame(model_summary_list, columns=['model', 'auc',
                                                                'auc_ci', 
                                                            #  'auc_ci_lower', 'auc_ci_upper', 
                                                             'f1_score',
                                                                'f1_score_ci' 
                                                            #  'f1_score_ci_lower', 'f1_score_ci_upper'
                                                             ])
difference_summary_df = pd.DataFrame(difference_summary_list, columns=['model1', 'model1_auc', 'model1_f1_score', 
                                                                       'model2', 'model2_auc', 'model2_f1_score', 
                                                                       'auc_diff', 
                                                                       'auc_diff_ci', 
                                                                    #    'auc_diff_ci_lower', 'auc_diff_ci_upper', 
                                                                       'auc_diff_p_value',  
                                                                       'f1_diff', 
                                                                       'f1_diff_ci',
                                                                       #'f1_diff_ci_lower', 'f1_diff_ci_upper', 
                                                                       'f1_diff_p_value'])

In [15]:
model_summary_df

,model,auc,auc_ci,f1_score,f1_score_ci
0,XGBoost Complete Data,0.976969,"[0.9659847076426393, 0.9865157488339849]",0.862069,"[0.8249400479616307, 0.8953771289537712]"
1,XGBoost Patient History,0.851167,"[0.8269613540556332, 0.8754519487330837]",0.413953,"[0.35714285714285715, 0.47139669826740765]"
2,Text Only Model,0.979403,"[0.9689956572476416, 0.9883371756600277]",0.858586,"[0.8219161813034849, 0.893827160493827]"
3,ConvNetV2 (Embeddings Split),0.991169,"[0.9847047587965302, 0.9958821417249051]",0.877805,"[0.8480392156862745, 0.9115290620269555]"
4,Image Only Model (Embeddings Split),0.948028,"[0.931060263722683, 0.9632095947375205]",0.714681,"[0.6707682926829268, 0.7683623516371866]"
5,Image Only Model,0.955765,"[0.938628007704196, 0.9701587390063791]",0.716157,"[0.672767838568754, 0.7644628099173554]"
6,Early Fusion Model,0.986939,"[0.9788420128666697, 0.9929160812455883]",0.864322,"[0.831894592476489, 0.9007273164149863]"
7,Late Fusion Model,0.987451,"[0.9782934883715608, 0.9942461431126883]",0.882927,"[0.8564088397790054, 0.9178960863697705]"
8,Attention Fusion Model,0.953751,"[0.9351729775169304, 0.9699845424383382]",0.729358,"[0.6819221967963387, 0.7726161369193153]"


In [16]:
difference_summary_df

,model1,model1_auc,model1_f1_score,model2,model2_auc,model2_f1_score,auc_diff,auc_diff_ci,auc_diff_p_value,f1_diff,f1_diff_ci,f1_diff_p_value
0,XGBoost Complete Data,0.976969,0.862069,XGBoost Patient History,0.851167,0.413953,-0.125802,"[-0.14940014754632824, -0.10200662983048735]",0.0000,-0.448115,"[-0.5126559036807256, -0.3814297399318367]",0.0000
1,XGBoost Complete Data,0.976969,0.862069,Text Only Model,0.979403,0.858586,0.002433,"[-0.002348591111167475, 0.008575878906036083]",0.1997,-0.003483,"[-0.027923089644701284, 0.022704022083398945]",0.4260
2,ConvNetV2 (Embeddings Split),0.991169,0.877805,Image Only Model (Embeddings Split),0.948028,0.714681,-0.043142,"[-0.058856008097651444, -0.028725784803901262]",0.0000,-0.163124,"[-0.20310268278296334, -0.11813915858506634]",0.0000
3,Image Only Model (Disjoint Test Split),0.955765,0.716157,Image Only Model (Embeddings Split),0.948028,0.714681,-0.007737,"[-0.03007527298872234, 0.015040769672915849]",0.2526,-0.001476,"[-0.06658048334193417, 0.06896505553492237]",0.5149
4,Image Only Model,0.955765,0.716157,Early Fusion Model,0.986939,0.864322,0.031174,"[0.018618301799342186, 0.04602862469105596]",0.0000,0.148164,"[0.10249678842873063, 0.1941932605316295]",0.0000
5,Image Only Model,0.955765,0.716157,Late Fusion Model,0.987451,0.882927,0.031686,"[0.01921511261011975, 0.046338556522108366]",0.0000,0.166770,"[0.1244988715483946, 0.2140627197027835]",0.0000
6,Image Only Model,0.955765,0.716157,Attention Fusion Model,0.953751,0.729358,-0.002014,"[-0.007936274928963311, 0.002850580769592822]",0.2460,0.013201,"[-0.012454188351179426, 0.029815970972005265]",0.2113
7,Early Fusion Model,0.986939,0.864322,Late Fusion Model,0.987451,0.882927,0.000512,"[-0.0021399628795698926, 0.003171630058934415]",0.3446,0.018605,"[0.0027734146347344313, 0.04043656655237824]",0.0116
